# List of Satellite with respective height and time of culmination pass

In [20]:
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime
import requests
from datetime import timedelta

In [21]:
from skyfield.api import Topos, load, EarthSatellite
from datetime import datetime
import requests


# Function to download TLE data from CelesTrak
def download_tle():
    url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    tle_lines = response.text.strip().splitlines()
    return tle_lines

# Load satellites using EarthSatellite class
def load_satellites(tle_lines):
    satellites = []
    for i in range(0, len(tle_lines), 3):
        name = tle_lines[i].strip()
        if "STARLINK" not in name.upper():  # Exclude Starlink satellites
            line1 = tle_lines[i + 1].strip()
            line2 = tle_lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name, load.timescale())
            satellites.append(satellite)
    return satellites

# Function to predict satellite culmination passes for a given location and time range
def predict_passes(satellites, location, start_time, end_time):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    results = []  # List to store results for sorting
    print(f"Predicting satellite culminations for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        for ti, event in zip(t, events):
            if event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                
                # Append result as a tuple (satellite name, time, altitude)
                results.append((satellite.name, ti.utc_strftime('%Y-%m-%d %H:%M:%S'), altitude))

    # Sort results by altitude
    results.sort(key=lambda x: x[2], reverse=True)  # Sort by altitude in descending order

    # Display sorted results
    for name, time, altitude in results:
        print(f"{name} culminate at {time} UTC, altitude: {altitude:.2f} km")

# Main function
if __name__ == "__main__":
    # Download and load TLE data
    #tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (47.3769, 8.5417)

    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 7, 17, 15, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 7, 17, 40, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite culmination passes
    predict_passes(satellites, location, start_time, end_time)


Predicting satellite culminations for location: (47.3769, 8.5417)
THOR 5 culminate at 2025-01-07 17:38:50 UTC, altitude: 38182.09 km
BEIDOU-2 IGSO-3 culminate at 2025-01-07 17:25:50 UTC, altitude: 38176.88 km
ARKTIKA-M 2 culminate at 2025-01-07 17:20:41 UTC, altitude: 28707.84 km
GSAT0203 (GALILEO 7) culminate at 2025-01-07 17:20:15 UTC, altitude: 23443.79 km
NAVSTAR 78 (USA 293) culminate at 2025-01-07 17:19:28 UTC, altitude: 20447.65 km
NAVSTAR 73 (USA 260) culminate at 2025-01-07 17:28:19 UTC, altitude: 20349.45 km
COSMOS 2522 [GLONASS-M] culminate at 2025-01-07 17:17:33 UTC, altitude: 19275.52 km
COSMOS 2545 [GLONASS-M] culminate at 2025-01-07 17:35:57 UTC, altitude: 19265.79 km
LAGEOS 1 culminate at 2025-01-07 17:30:05 UTC, altitude: 7197.85 km
LAGEOS 2 culminate at 2025-01-07 17:25:29 UTC, altitude: 6001.22 km
EXPRESS-MD2 culminate at 2025-01-07 17:21:28 UTC, altitude: 2586.05 km
GLOBALSTAR M085 culminate at 2025-01-07 17:18:40 UTC, altitude: 2238.58 km
COSMOS 2386 culminate at 2

In [25]:
# Function to download TLE data from CelesTrak
def download_tle():
    url = 'https://celestrak.org/NORAD/elements/gp.php?GROUP=active&FORMAT=tle'
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    tle_lines = response.text.strip().splitlines()
    return tle_lines

# Load satellites using EarthSatellite class (only GLOBALSTAR satellites)
def load_satellites(tle_lines):
    satellites = []
    for i in range(0, len(tle_lines), 3):
        name = tle_lines[i].strip()
        if "GLOBALSTAR" in name.upper():  # Include only GLOBALSTAR satellites
            line1 = tle_lines[i + 1].strip()
            line2 = tle_lines[i + 2].strip()
            satellite = EarthSatellite(line1, line2, name, load.timescale())
            satellites.append(satellite)
    return satellites

# Function to predict satellite passes (rise, culmination, set) for a given location and time range
def predict_passes(satellites, location, start_time, end_time):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    results = []  # List to store results for sorting
    print(f"Predicting satellite passes for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        pass_info = {"name": satellite.name, "rise": None, "culminate": None, "set": None}

        for ti, event in zip(t, events):
            # Convert the Skyfield time object to a datetime and apply UTC+1 offset
            #ti_dt = ti.utc_datetime() + timedelta(hours=1)

            if event == 0:  # 0 corresponds to 'rise'
                pass_info["rise"] = ti.utc_strftime('%Y-%m-%d %H:%M:%S')
            elif event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                pass_info["culminate"] = (ti.utc_strftime('%Y-%m-%d %H:%M:%S'), altitude)
            elif event == 2:  # 2 corresponds to 'set'
                pass_info["set"] = ti.utc_strftime('%Y-%m-%d %H:%M:%S')

        # Append result only if all three events were found
        if pass_info["rise"] and pass_info["culminate"] and pass_info["set"]:
            results.append(pass_info)

    # Display results
    for pass_info in results:
        rise = pass_info["rise"]
        culmination_time, altitude = pass_info["culminate"]
        set_time = pass_info["set"]
        print(f"{pass_info['name']}:\n"
              f"  Rise:       {rise} UTC\n"
              f"  Culminate:  {culmination_time} UTC, altitude: {altitude:.2f} km\n"
              f"  Set:        {set_time} UTC\n")

from datetime import timedelta

# Function to predict satellite passes (rise, culmination, set) for a given location and time range
def predict_passes(satellites, location, start_time, end_time):
    ts = load.timescale()
    observer = Topos(latitude_degrees=location[0], longitude_degrees=location[1])

    results = []  # List to store results for sorting
    print(f"Predicting satellite passes for location: {location}")
    for satellite in satellites:
        t, events = satellite.find_events(observer, start_time, end_time, altitude_degrees=30.0)
        pass_info = {"name": satellite.name, "rise": None, "culminate": None, "set": None}

        for ti, event in zip(t, events):
            # Convert the Skyfield time object to a datetime and apply UTC+1 offset
            ti_dt = ti.utc_datetime() + timedelta(hours=1)

            if event == 0:  # 0 corresponds to 'rise'
                pass_info["rise"] = ti_dt.strftime('%Y-%m-%d %H:%M:%S')
            elif event == 1:  # 1 corresponds to 'culminate'
                # Calculate satellite altitude during culmination
                difference = satellite - observer
                topocentric = difference.at(ti)
                altitude = topocentric.distance().km  # Altitude in kilometers
                pass_info["culminate"] = (ti_dt.strftime('%Y-%m-%d %H:%M:%S'), altitude)
            elif event == 2:  # 2 corresponds to 'set'
                pass_info["set"] = ti_dt.strftime('%Y-%m-%d %H:%M:%S')

        # Append result only if all three events were found
        if pass_info["rise"] and pass_info["culminate"] and pass_info["set"]:
            results.append(pass_info)

    # Display results
    for pass_info in results:
        rise = pass_info["rise"]
        culmination_time, altitude = pass_info["culminate"]
        set_time = pass_info["set"]
        print(f"{pass_info['name']}:\n"
              f"  Rise:       {rise} UTC+1\n"
              f"  Culminate:  {culmination_time} UTC+1, altitude: {altitude:.2f} km\n"
              f"  Set:        {set_time} UTC+1\n")

from datetime import datetime, timedelta

# Main function
if __name__ == "__main__":
    # Download and load TLE data
    #tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (46.410556, 9.967222)

    # Set prediction time range in UTC+1 (user-friendly input)
    start_time_utc1 = datetime(2025, 1, 7, 16, 30, 0)  # 8 Jan 2025, 07:00 UTC+1
    end_time_utc1 = datetime(2025, 1, 7, 18, 0, 0)    # 8 Jan 2025, 08:00 UTC+1

    # Convert UTC+1 time to UTC by subtracting 1 hour
    start_time_utc = start_time_utc1 - timedelta(hours=1)
    end_time_utc = end_time_utc1 - timedelta(hours=1)

    # Use Skyfield to create time objects in UTC
    ts = load.timescale()
    start_time = ts.utc(start_time_utc.year, start_time_utc.month, start_time_utc.day,
                        start_time_utc.hour, start_time_utc.minute, start_time_utc.second)
    end_time = ts.utc(end_time_utc.year, end_time_utc.month, end_time_utc.day,
                      end_time_utc.hour, end_time_utc.minute, end_time_utc.second)

    # Predict satellite passes
    predict_passes(satellites, location, start_time, end_time)


""""
# Main function
if __name__ == "__main__":
    # Download and load TLE data
    #tle_lines = download_tle()
    satellites = load_satellites(tle_lines)

    # Set location for Zurich, Switzerland (latitude, longitude)
    location = (46.410556, 9.967222)

    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)
    ts = load.timescale()
    start_time = ts.utc(2025, 1, 8, 6, 00, 0)  # 8 Jan 2025, 01:00 UTC
    end_time = ts.utc(2025, 1, 8, 7, 00, 0)    # 8 Jan 2025, 02:00 UTC

    # Predict satellite passes
    predict_passes(satellites, location, start_time, end_time)
    """

Predicting satellite passes for location: (46.410556, 9.967222)
GLOBALSTAR M069:
  Rise:       2025-01-07 17:49:41 UTC+1
  Culminate:  2025-01-07 17:54:45 UTC+1, altitude: 1455.84 km
  Set:        2025-01-07 17:59:51 UTC+1

GLOBALSTAR M079:
  Rise:       2025-01-07 17:11:37 UTC+1
  Culminate:  2025-01-07 17:16:14 UTC+1, altitude: 1582.15 km
  Set:        2025-01-07 17:20:52 UTC+1

GLOBALSTAR M076:
  Rise:       2025-01-07 16:31:19 UTC+1
  Culminate:  2025-01-07 16:35:58 UTC+1, altitude: 1573.22 km
  Set:        2025-01-07 16:40:37 UTC+1

GLOBALSTAR M075:
  Rise:       2025-01-07 16:41:57 UTC+1
  Culminate:  2025-01-07 16:46:43 UTC+1, altitude: 1496.04 km
  Set:        2025-01-07 16:51:29 UTC+1

GLOBALSTAR M073:
  Rise:       2025-01-07 17:00:42 UTC+1
  Culminate:  2025-01-07 17:05:31 UTC+1, altitude: 1476.43 km
  Set:        2025-01-07 17:10:20 UTC+1

GLOBALSTAR M092:
  Rise:       2025-01-07 17:40:48 UTC+1
  Culminate:  2025-01-07 17:45:05 UTC+1, altitude: 1677.31 km
  Set:        202

'"\n# Main function\nif __name__ == "__main__":\n    # Download and load TLE data\n    #tle_lines = download_tle()\n    satellites = load_satellites(tle_lines)\n\n    # Set location for Zurich, Switzerland (latitude, longitude)\n    location = (46.410556, 9.967222)\n\n    # Set prediction time range (8 Jan 2025, 01:00 UTC to 8 Jan 2025, 02:00 UTC)\n    ts = load.timescale()\n    start_time = ts.utc(2025, 1, 8, 6, 00, 0)  # 8 Jan 2025, 01:00 UTC\n    end_time = ts.utc(2025, 1, 8, 7, 00, 0)    # 8 Jan 2025, 02:00 UTC\n\n    # Predict satellite passes\n    predict_passes(satellites, location, start_time, end_time)\n    '